In [13]:
%matplotlib inline
import numpy as np
import pandas as pd
from math import floor
from scipy.spatial import distance

def read_pdb(filename):
    with open(filename, 'r') as file:
        strline_L = file.readlines()
    X_list = list()
    Y_list = list()
    Z_list = list()
    atomtype_list = list()
    for strline in strline_L:
        # removes all whitespace at the start and end, including spaces, tabs, newlines and carriage returns
        stripped_line = strline.strip()

        line_length = len(stripped_line)
        # print("Line length:{}".format(line_length))
        if line_length < 78:
            print("ERROR: line length is different. Expected>=78, current={}".format(line_length))
        X_list.append(float(stripped_line[30:38].strip()))
        Y_list.append(float(stripped_line[38:46].strip()))
        Z_list.append(float(stripped_line[46:54].strip()))

        atomtype = stripped_line[76:78].strip()
        if atomtype == 'C':
            atomtype_list.append('h') # 'h' means hydrophobic
        else:
            atomtype_list.append('p') # 'p' means polar

    return X_list, Y_list, Z_list, atomtype_list

#### Set params

In [5]:
x_lower = -50
x_upper = 50
y_lower = -50
y_upper = 50
z_lower = -50
z_upper = 50
unit = 1

def point_within_vox(pt, x_lower=-50, x_upper=50, y_lower=-50, y_upper=50, z_lower=-50, z_upper=50):
    (atom_x, atom_y, atom_z) = pt
    check_x = (atom_x>=x_lower) and (atom_x<x_upper)
    check_y = (atom_y>=y_lower) and (atom_y<y_upper)
    check_z = (atom_z>=z_lower) and (atom_z<z_upper)
    return check_x and check_y and check_z

In [8]:
X_list, Y_list, Z_list, atomtype_list=read_pdb("training_data/0001_pro_cg.pdb")
all_atoms = list(zip(X_list, Y_list, Z_list, atomtype_list))
print(len(all_atoms))
use_atoms = [x for x in all_atoms if point_within_vox((x[0], x[1], x[2]))]
print(len(use_atoms))

701
609


In [27]:
def convert_xyz_to_vox(atoms_list, 
                       x_lower=-50, x_upper=50, y_lower=-50, 
                       y_upper=50, z_lower=-50, z_upper=50, unit=1): 
    width = x_upper-x_lower
    vox_h = x = np.zeros((width, width, width))
    vox_p = x = np.zeros((width, width, width))
    
    for atom in use_atoms:
        (x, y, z, t) = atom
        index_x = floor((x-x_lower)/unit)
        index_y = floor((y-y_lower)/unit)
        index_z = floor((z-z_lower)/unit)
        if t =='h':
            vox_h[index_z][index_x, index_y] += 1
        elif t =='p':
            vox_p[index_z][index_x, index_y] += 1

    return (vox_h, vox_p)

In [28]:
(vox_h, vox_p) = convert_xyz_to_vox(use_atoms)

In [30]:
vox_h.shape

(100, 100, 100)

In [22]:
test = np.zeros((5,5,5))

In [25]:
test[4][2, 3] = 1

In [26]:
test[4]

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])